In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import torch
import torch_geometric as pyg

In [3]:
from smartgd.model import Generator
from smartgd.data import GraphDrawingData
from smartgd.datasets import  RomeDataset

In [4]:
GraphDrawingData.set_optional_fields(["edge_pair_metaindex", "face", "rng"])
dataset = RomeDataset(
    index=pd.read_csv("assets/rome_index.txt", header=None)[0],
)

Transform graphs:   0%|          | 0/11531 [00:00<?, ?it/s]

In [5]:
params = Generator.Params(
    num_blocks=10,
    block_depth=2,
    block_width=8,
    block_output_dim=8,
    edge_net_depth=1,
    edge_net_width=16,
    edge_attr_dim=2,
    node_attr_dim=2,
)

In [6]:
generator = Generator(
    params=params,
)

In [7]:
loader = pyg.loader.DataLoader(dataset, batch_size=8)

In [8]:
batch = next(iter(loader))
batch

GraphDrawingDataBatch(G=[8], perm_index=[2, 13554], edge_metaindex=[784], apsp_attr=[13554], perm_weight=[13554], aggr_metaindex=[13554], pos=[307, 2], name=[8], n=[8], m=[8], edge_pair_metaindex=[2, 11403], num_nodes=307, batch=[307], ptr=[9])

In [9]:
batch.perm_index.min(), batch.perm_index.max()

(tensor(0), tensor(306))

In [14]:
generator(
    init_pos=batch.pos,
    edge_index=batch.perm_index,
    edge_attr=torch.cat([batch.apsp_attr[:, None], 1 / batch.apsp_attr[:, None].square()], dim=-1),
    batch_index=batch.batch,
)

(tensor([[ 1.8338e+01, -2.1582e+01],
         [ 4.4677e+00, -3.7498e-01],
         [ 8.2080e+00, -1.7034e+01],
         [ 2.3217e+01, -2.2530e+01],
         [ 2.1426e+01, -1.1136e+01],
         [ 1.6561e+00, -6.2770e+00],
         [ 9.5271e+00, -5.5295e+00],
         [ 6.1212e+00,  2.1450e+00],
         [ 1.5078e+01, -2.0388e+01],
         [ 1.7563e+01, -1.6277e+01],
         [ 1.8581e+01, -5.2127e+00],
         [ 1.1312e+01, -7.1986e-01],
         [ 1.2330e+01, -8.1074e+00],
         [ 1.4354e+01, -7.1701e+00],
         [ 7.7865e+00, -7.1980e+00],
         [ 1.8717e+01, -1.2021e+01],
         [-1.8471e+00, -2.5332e+00],
         [ 4.2906e-01, -9.0235e-01],
         [-1.3388e+00, -2.0202e+00],
         [-4.0195e-01, -2.6613e+00],
         [-1.1411e+00, -2.7556e+00],
         [-7.7511e-01, -3.0052e+00],
         [-9.5194e-01, -1.5371e+00],
         [ 1.5340e-01, -1.8054e+00],
         [-7.6379e-01, -2.1409e+00],
         [ 1.4032e-02, -2.7154e+00],
         [-6.4786e-01, -1.6996e+00],
 